# LGBM + CV + training

## Load parameters

In [ ]:
import glob

param_list = glob.glob("optuna_lgbm.csv")
models = list()
best_lgbm_params = list()

lgbm_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if lgbm_params.shape[0] == 0:
        lgbm_params = tmp
    else:
        lgbm_params = pd.concat([lgbm_params, tmp])
        
lgbm_params = lgbm_params.sort_values('value').head(20)
param_cols = [c for c in lgbm_params.columns if c.startswith('params_')]
lgbm_params = lgbm_params[param_cols]

for idx, row in lgbm_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['objective'] = 'binary'
    row_dict['metric'] = 'none'
#     row_dict['subsample_for_bin'] = 300000
    row_dict['force_col_wise'] = False
    row_dict['early_stopping_rounds'] = 50
    row_dict['verbose'] = -1
    row_dict['max_bin'] = 255
    row_dict['bagging_freq'] = int(row_dict['bagging_freq'])
    row_dict['min_data_in_leaf'] = int(row_dict['min_data_in_leaf'])
    row_dict['n_estimators'] = 3000 # int(row_dict['n_estimators'])
    row_dict['learning_rate'] = float(row_dict['learning_rate'])
    row_dict['num_leaves'] = int(row_dict['num_leaves'])
    row_dict['max_depth'] = int(row_dict['max_depth'])
    row_dict['is_unbalance'] = True
    row_dict['class_weight'] = 'balanced'
    row_dict['verbose'] = -1
    
    if row_dict['boosting_type'] == 'goss':
        row_dict['bagging_fraction'] = None
        
    best_lgbm_params.append(row_dict)

## Train with the Cross Validation + RandomUnderSampling

In [ ]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def lgbm_training():
    # Make random under-sampling to balance classes
    positive_count_train = train_df['Class'].value_counts()[1]
    sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 
                                                    1: positive_count_train}, 
                                 random_state=15062021, 
                                 replacement=True)

    X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']

    if CFG.undersample:
        X_re, y_re = sampler.fit_resample(X_re, y_re)
    
    kf = MultilabelStratifiedKFold(n_splits=CFG.n_stacking_folds, shuffle=True, random_state=8062023+20)

    oof_level2 = np.zeros([y_re.shape[0], len(best_lgbm_params) + 1])
    oof_level2[:, len(best_lgbm_params)] = y_re
    oof_level2_test = np.zeros([test_df.shape[0], len(best_lgbm_params)])

    print(f"Training with {blu}{X_re.shape[1]}{res} features")

    for fold, (fit_idx, val_idx) in tqdm(enumerate(kf.split(X=X_re, y=X_re.iloc[:,-3:]), start = 1),
                                         total=CFG.n_stacking_folds):
        X, y, test = X_re[features], y_re, test_df[features]
        
        # Split the dataset according to the fold indexes.
        X_train = X.iloc[fit_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[fit_idx]
        y_val = y.iloc[val_idx]

        for i, params in enumerate(best_lgbm_params):
            
            clf = lgb.LGBMClassifier(**params)
            clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                    eval_metric=bll_metric, verbose=-1)

            val_preds = clf.predict_proba(X_val)[:,1]
            oof_level2[val_idx, i] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            best_iter = clf.best_iteration_

            print(clf.best_iteration_)
            
            print(f'Fold: {blu}{fold:>3}{res}| bll_metric: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')
            
            oof_level2_test[:, i] += clf.predict_proba(test)[:,1]
        
    return oof_level2, oof_level2_test / CFG.n_stacking_folds

if CFG.stacking:
    oof_level2_lgbm, oof_level2_test_lgbm = lgbm_training()

# CatBoost + CV + training

## Load parameters

In [ ]:
import glob

param_list = glob.glob("optuna_catboost.csv")
models = list()
best_cb_params = list()

cb_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if cb_params.shape[0] == 0:
        cb_params = tmp
    else:
        cb_params = pd.concat([cb_params, tmp])
        
cb_params = cb_params.sort_values('value').head(10)
param_cols = [c for c in cb_params.columns if c.startswith('params_')]
cb_params = cb_params[param_cols]


for idx, row in cb_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['task_type'] = 'CPU'
    row_dict['auto_class_weights'] = 'Balanced'
    row_dict['eval_metric'] = 'Logloss'
    row_dict['loss_function'] = 'Logloss'
    row_dict['random_seed'] = 13062023
    row_dict['verbose'] = 0
    row_dict['od_type'] = 'Iter'
    row_dict['od_wait'] = 100
    row_dict['border_count'] = 254
    row_dict['iterations'] = 3000
    row_dict['learning_rate'] = float(row_dict['learning_rate'])
    row_dict['l2_leaf_reg'] = float(row_dict['l2_leaf_reg'])
    row_dict['depth'] = int(row_dict['depth'])
    row_dict['random_strength'] = float(row_dict['random_strength'])
    row_dict['min_data_in_leaf'] = int(row_dict['min_data_in_leaf'])
    
    if row_dict["bootstrap_type"] == "Bayesian":
        row_dict['bagging_temperature'] = float(row_dict['bagging_temperature'])
    else:    
        row_dict['bagging_temperature'] = None
        
    if row_dict["bootstrap_type"] == "Bernoulli":
        row_dict['subsample'] = float(row_dict['subsample'])
    else:
        row_dict['subsample'] = None
    
    if row_dict['grow_policy'] == 'Lossguide':
        row_dict['max_leaves'] = int(row_dict['max_leaves'])
    else:
        row_dict['max_leaves'] = None
    
    if row_dict['grow_policy'] != 'SymmetricTree':
        row_dict['boosting_type'] = 'Plain'
    
    best_cb_params.append(row_dict)

## Train with the Cross Validation + RandomUnderSampling

In [ ]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def cb_training():
    # Make random under-sampling to balance classes
    positive_count_train = train_df['Class'].value_counts()[1]
    sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 
                                                    1: positive_count_train}, 
                                 random_state=15062021, 
                                 replacement=True)

    X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']

    if CFG.undersample:
        X_re, y_re = sampler.fit_resample(X_re, y_re)
    
    kf = MultilabelStratifiedKFold(n_splits=CFG.n_stacking_folds, shuffle=True, random_state=8062023+20)

    oof_level2 = np.zeros([y_re.shape[0], len(best_cb_params) + 1])
    oof_level2[:, len(best_cb_params)] = y_re
    oof_level2_test = np.zeros([test_df.shape[0], len(best_cb_params)])

    print(f"Training with {blu}{X_re.shape[1]}{res} features")

    for fold, (fit_idx, val_idx) in tqdm(enumerate(kf.split(X=X_re, y=X_re.iloc[:,-3:]), start = 1),
                                         total=CFG.n_stacking_folds):
        X, y, test = X_re[features], y_re, test_df[features]
        
        # Split the dataset according to the fold indexes.
        X_train = X.iloc[fit_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[fit_idx]
        y_val = y.iloc[val_idx]

        train_pool = Pool(X_train, y_train, cat_features=['EJ'])
        val_pool = Pool(X_val, y_val, cat_features=['EJ'])
        
        for i, params in enumerate(best_cb_params):
            
            model = cat.CatBoostClassifier(**params)
            model.fit(train_pool, eval_set=val_pool, verbose=0)

            val_preds = model.predict_proba(val_pool)[:,1]
            oof_level2[val_idx, i] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            best_iter = model.best_iteration_

            print(model.best_iteration_)
            
            print(f'Fold: {blu}{fold:>3}{res}| bll_metric: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')
        
            oof_level2_test[:, i] += model.predict_proba(test)[:,1]
        
    return oof_level2, oof_level2_test / CFG.n_stacking_folds

if CFG.stacking:
    oof_level2_cb, oof_level2_test_cb = cb_training()

# XGBoost + CV + training

## Load parameters

In [ ]:
import glob

param_list = glob.glob("optuna_xgb.csv")
models = list()
best_xb_params = list()

xb_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if xb_params.shape[0] == 0:
        xb_params = tmp
    else:
        xb_params = pd.concat([cb_params, tmp])
        
xb_params = xb_params.sort_values('value').head(10)
param_cols = [c for c in xb_params.columns if c.startswith('params_')]
xb_params = xb_params[param_cols]

for idx, row in xb_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['n_estimators'] = 3000
    row_dict['random_state'] = 14062023
    row_dict['early_stopping_rounds'] = 100
    row_dict['verbosity'] = 0
    row_dict['scale_pos_weight'] = 4.71
    row_dict['objective'] = "binary:logistic"
    row_dict['eval_metric'] = "logloss"
    row_dict['tree_method'] = "exact"
    row_dict['booster'] = "gbtree"

    if row_dict["booster"] in ["gbtree", "dart"]:
        row_dict["learning_rate"] = float(row_dict['learning_rate'])
        row_dict["max_depth"] = int(row_dict['max_depth'])
        row_dict["min_child_weight"] = float(row_dict['min_child_weight'])
        row_dict["gamma"] = float(row_dict['gamma'])
    else:
        row_dict["learning_rate"] = None
        row_dict["max_depth"] = None
        row_dict["min_child_weight"] = None
        row_dict["gamma"] = None
        row_dict["grow_policy"] = None     

    if row_dict["booster"] == "dart":
        row_dict["rate_drop"] = float(row_dict['rate_drop'])
        row_dict["skip_drop"] = float(row_dict['skip_drop'])
    else:
        row_dict["sample_type"] = None
        row_dict["normalize_type"] = None
        row_dict["rate_drop"] = None
        row_dict["skip_drop"] = None

    best_xb_params.append(row_dict)

## Train with the Cross Validation + RandomUnderSampling

In [ ]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def xgboost_training():
    # Make random under-sampling to balance classes
    positive_count_train = train_df['Class'].value_counts()[1]
    sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 
                                                    1: positive_count_train}, 
                                 random_state=15062020, 
                                 replacement=True)

    X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']

    if CFG.undersample:
        X_re, y_re = sampler.fit_resample(X_re, y_re)
    
    kf = MultilabelStratifiedKFold(n_splits=CFG.n_stacking_folds, shuffle=True, random_state=8062023+20)

    oof_level2 = np.zeros([y_re.shape[0], len(best_xb_params) + 1])
    oof_level2[:, len(best_xb_params)] = y_re
    oof_level2_test = np.zeros([test_df.shape[0], len(best_xb_params)])

    print(f"Training with {blu}{X_re.shape[1]}{res} features")

    for fold, (fit_idx, val_idx) in tqdm(enumerate(kf.split(X=X_re, y=X_re.iloc[:,-3:]), start = 1),
                                         total=CFG.n_stacking_folds):
        X, y, test = X_re[features], y_re, test_df[features]
        
        # Split the dataset according to the fold indexes.
        X_train = X.iloc[fit_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[fit_idx]
        y_val = y.iloc[val_idx]

        for i, params in enumerate(best_xb_params):
            clf = xgb.XGBClassifier(**params)
            
            clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=500)

            val_preds = clf.predict_proba(X_val)[:,1]
            oof_level2[val_idx, i] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            best_iter = 0

#             print(clf.best_iteration_)
            
            print(f'Fold: {blu}{fold:>3}{res}| bll_metric: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')
        
            oof_level2_test[:, i] += clf.predict_proba(test)[:,1]
        
    return oof_level2, oof_level2_test / CFG.n_stacking_folds

if CFG.stacking:
    oof_level2_xgb, oof_level2_test_xgb = xgboost_training()

# Stacking LGBM + CatBoost + XGBoost with Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

oof_level2 = np.concatenate([oof_level2_lgbm[:,:-1] , oof_level2_cb[:,:-1], oof_level2_xgb[:,:-1]], axis=1)
oof_level2_test = np.concatenate([oof_level2_test_lgbm , oof_level2_test_cb, oof_level2_test_xgb], axis=1)

X = oof_level2
y = oof_level2_lgbm[:,-1]

# mean bll
print(balanced_log_loss(y, np.mean(X, axis=1)))

lr = LogisticRegression(class_weight='balanced')
lr.fit(X, y)

pred = lr.predict_proba(X)[:,1]

# lr bll
print(balanced_log_loss(y, pred))

weights = lr.coef_[0]

# LGBM + TSS + Stacking

## Create metafeatures for the train set

In [ ]:
# dates for the second level
dates_level2 = [28, 29, 30, 31, 32, 33]

# train dates
dates_train = X_train.date_block_num
# dates_train_level2 = dates_train[dates_train.isin(dates)]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin(dates_level2)]

# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], len(best_params)+1])
X_train_level2[:, len(best_params)] = y_train_level2

meta_index_begin = 0
meta_index_end = 0

# Now fill `X_train_level2` with metafeatures
for cur_block_num in tqdm_notebook(dates_level2):
    
    # split data
    train_index = X_train.loc[dates_train <  cur_block_num].index
    test_index  = X_train.loc[dates_train == cur_block_num].index
    
    X_train_l2 = X_train.loc[train_index, :]
    X_test_l2 =  X_train.loc[test_index, :]

    y_train_l2 = y_train[train_index]
    y_test_l2 =  y_train[test_index]
    
    meta_index_end += y_test_l2.shape[0]
    
    # predict metafeatures for each of LGBM regressors
    for i, params in enumerate(tqdm_notebook(best_params)):
        lgb = LGBMRegressor(**params)
        reg = lgb.fit(X_train_l2, y_train_l2)
        pred = lgb.predict(X_test_l2)
        X_train_level2[meta_index_begin:meta_index_end, i] = pred

        del lgb, reg, pred
        gc.collect()
        
    meta_index_begin = meta_index_end
        
X_train_level2 = pd.DataFrame(X_train_level2, columns=[f'lgbr_{i+1}' for i in range(len(best_params))]+['gt'])
X_train_level2.to_pickle('LGBM_X_train_level2.pkl')

## Create metafeatures for the test set

In [ ]:
X_test_level2 = np.zeros([X_test.shape[0], len(best_params)])

for i, params in enumerate(tqdm_notebook(best_params)):
    lgb = LGBMRegressor(**params)
    reg = lgb.fit(X_train, y_train)
    pred = lgb.predict(X_test)
    X_test_level2[:, i] = pred
    
    del lgb, reg, pred
    gc.collect()
    
X_test_level2 = pd.DataFrame(X_test_level2, columns=[f'lgbr_{i+1}' for i in range(len(best_params))])
X_test_level2.to_pickle('LGBM_X_test_level2.pkl')

## Stacking with Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

X = X_train_level2[[c for c in X_train_level2.columns if c != 'gt']]
y = X_train_level2['gt']

lr = LinearRegression()
lr.fit(X, y)

pred = lr.predict(X_test_level2)